In [1]:
from polarspark import SparkSession
from polarspark import functions as F

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
data = [('James','','Smith','1991-04-01','M',3000),
  ('Michael','Rose','','2000-05-19','M',4000),
  ('Robert','','Williams','1978-09-05','M',4000),
  ('Maria','Anne','Jones','1967-12-01','F',4000),
  ('Jen','Mary','Brown','1980-02-17','F',-1)
]

columns = ["firstname","middlename","lastname","dob","gender","salary"]
df = spark.createDataFrame(data=data, schema = columns)

/workspace/polarspark/sql/session.py:28: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  df = pl.from_records(data, schema=schema)


In [4]:
df.show()

shape: (5, 6)
┌───────────┬────────────┬──────────┬────────────┬────────┬────────┐
│ firstname ┆ middlename ┆ lastname ┆ dob        ┆ gender ┆ salary │
│ ---       ┆ ---        ┆ ---      ┆ ---        ┆ ---    ┆ ---    │
│ str       ┆ str        ┆ str      ┆ str        ┆ str    ┆ i64    │
╞═══════════╪════════════╪══════════╪════════════╪════════╪════════╡
│ James     ┆            ┆ Smith    ┆ 1991-04-01 ┆ M      ┆ 3000   │
│ Michael   ┆ Rose       ┆          ┆ 2000-05-19 ┆ M      ┆ 4000   │
│ Robert    ┆            ┆ Williams ┆ 1978-09-05 ┆ M      ┆ 4000   │
│ Maria     ┆ Anne       ┆ Jones    ┆ 1967-12-01 ┆ F      ┆ 4000   │
│ Jen       ┆ Mary       ┆ Brown    ┆ 1980-02-17 ┆ F      ┆ -1     │
└───────────┴────────────┴──────────┴────────────┴────────┴────────┘


In [13]:
(
    df.alias("a")
    .join(
        df.alias("b"), 
        on=[F.col("a.firstname") == F.col("b.firstname")], 
        how="left"
    )
    .groupBy(F.col("a.gender"))
    .agg(
        F.array_distinct(F.collect_list(
            F.struct(
                F.col("a.firstname").alias("first_name"),
                F.col("a.lastname").alias("last_name"), 
                F.col("a.salary").alias("salary")
            )
        )).alias("persons")    
    )
    .select(
        "gender",
        F.transform(
            "persons", 
            lambda x: F.struct(
                F.concat(x.first_name, " ", x.last_name).alias("full_name"), 
                x.salary.alias("salary"))
            ).alias("persons"),
        F.size("persons").alias("person_count")
    )

).show(truncate=False)

shape: (2, 3)
┌────────┬─────────────────────────────────┬──────────────┐
│ gender ┆ persons                         ┆ person_count │
│ ---    ┆ ---                             ┆ ---          │
│ str    ┆ list[struct[2]]                 ┆ u32          │
╞════════╪═════════════════════════════════╪══════════════╡
│ F      ┆ [{"Maria Jones",4000}, {"Jen B… ┆ 2            │
│ M      ┆ [{"James Smith",3000}, {"Micha… ┆ 3            │
└────────┴─────────────────────────────────┴──────────────┘
